In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.42B.300d'
max_size = 64
min_freq = 5
batch_size = 64

In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, max_size, min_freq, batch_size)

In [4]:
# AttractiveData.LABEL.vocab.freqs

In [5]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['Sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'Harvey', 'Nichols', "'", 'hilarious', 'Christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '3.3333333333333335'} {'Headline': ['Three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [19]:
AttractiveData.CATEGORIES_LABEL.vocab.freqs

Counter({'travel': 283,
         'health': 289,
         'femail': 291,
         'sport': 11,
         'gardening': 10,
         'sciencetech': 292,
         'news': 291,
         'food': 208,
         'football': 220,
         'travelnews': 9,
         'cricket': 7,
         'golf': 5,
         'books': 66,
         'rugbyunion': 14,
         'home': 7,
         'boxing': 4,
         'tennis': 13,
         'concussion': 1,
         'othersports': 12,
         'beauty': 1,
         'formulaone': 4,
         'racing': 2})

In [7]:
# a = next(iter(AttractiveData.testloader))
# for i in range(len(a)):
#     print(a.Headline[i], a.Category[i])
#     1/0

## Start to train

In [8]:
timestr = time.strftime("%Y%m%d-%H%M%S")
save_name = './model/AttractiveNet'
num_workers = 10
input_dim = len(AttractiveData.TEXT.vocab)
embedding_dim = 300

category_dim = len(AttractiveData.CATEGORIES_LABEL.vocab)
category_output_dim = 16
hidden_dim = 256
output_dim = 1
log_steps = 10
epochs = 100
lr = 1e-3
num_layers = 3
nhead = 4
dropout = 0.1
pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1410, 300])


In [9]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [10]:
AttractiveTrainer = AttractiveTrainer(save_name, log_steps, epochs, lr, timestr, AttractiveData.device, AttractiveData.trainloader, input_dim, category_dim, category_output_dim, embedding_dim, hidden_dim, output_dim, pretrained_embeddings, dropout, num_layers, nhead)

In [11]:
AttractiveTrainer.model

TransformerModel(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1410, 300, padding_idx=0)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(22, 16, padding_idx=0)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn)

In [12]:
AttractiveTrainer.train()

Epoch 0
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 155.93it/s]

EP_train | avg_loss: 1.5890047065913677 |
Epoch 1
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 156.98it/s]

EP_train | avg_loss: 0.7881287876516581 |
Epoch 2
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 157.19it/s]

EP_train | avg_loss: 0.7345287501811981 |
Epoch 3
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 150.67it/s]

EP_train | avg_loss: 0.7286438476294279 |
Epoch 4
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 154.28it/s]

EP_train | avg_loss: 0.7130521778017282 |
Epoch 5
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 155.76it/s]

EP_train | avg_loss: 0.7084342855960131 |
Epoch 6
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 157.17it/s]

EP_train | avg_loss: 0.6933034658432007 |
Epoch 7
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 156.36it/s]

EP_train | avg_loss: 0.6839309819042683 |
Epoch 8
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 155.96it/s]

EP_train | avg_loss: 0.67

## Below is testing

In [13]:
from transformermodel import TransformerModel
PATH = './model/AttractiveNet_20201029-114109_0.564.100'
load_model = TransformerModel(nhead, input_dim, category_dim, category_output_dim, embedding_dim, hidden_dim, output_dim, dropout, num_layers).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

TransformerModel(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1410, 300, padding_idx=0)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(22, 16, padding_idx=0)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn)

In [30]:
def predict_attractive(sentence, category):
    if category == 'living':
        category = 'home'
    if category == 'middleeast':
        category = 'news'
    if category == 'us':
        category = 'news'
    
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)
    tensor_category = tensor_category
    prediction = load_model(tensor_sentence, tensor_category)
    
    return prediction

In [31]:
predict_list = []
for i, sentence in enumerate(AttractiveData.test_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    predict_list.append(prediction.item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv('transformers.csv', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226


In [28]:
train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
for each_test in test_category:
    if each_test not in train_category:
        print(each_test)

living
middleeast
us


In [29]:
train_category

['travel',
 'health',
 'femail',
 'sport',
 'gardening',
 'sciencetech',
 'news',
 'food',
 'football',
 'travelnews',
 'cricket',
 'golf',
 'books',
 'rugbyunion',
 'home',
 'boxing',
 'tennis',
 'concussion',
 'othersports',
 'beauty',
 'formulaone',
 'racing']

In [25]:
test_headline = AttractiveData.df_test['Headline'].to_list()
predict_list = []
for i in range(len(test_headline)):
    predict_list.append(predict_attractive(test_headline[i]).item())
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv('transformers.csv', index=False)

## Below just for fun guess

In [17]:
a = AttractiveData.df_test['ID'].to_list()

In [17]:
from sklearn.metrics import mean_squared_error

In [9]:
mean_squared_error(a, b)
# Training all 3.0 got mse = 0.5545

0.5545343137254902

In [18]:
# only for fun
import random
guess_list = [2.3333333333333335, 3.3333333333333335, 3.6666666666666665, 2.6666666666666665]
b = []
for i in range(len(a)):
    b.append(random.choice(guess_list))

In [19]:
AttractiveData.df_test['Label'] = b

In [20]:
AttractiveData.df_test[['ID', 'Label']].to_csv('all_3.csv', index=False)